In [353]:
import numpy as np
import json
import os
import pandas as pd

In [354]:
#Toma los archivos .json del directorio y los une a una base de datos usando pandas

data_list = []
base_dir = 'tech---ops/'

for file in os.listdir(base_dir):
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, dtype=False)
        data_list.append(json_data)

df = pd.concat(data_list)


In [355]:
# Se agregaron las columnas nuevas segun las reacciones y ademas se ordenaron.

nuevas_columnas_reacciones = ['t_finalizado', 't_revisado', 't_resuelto']

df_columnas_reaccion = df.reindex(columns=df.columns.tolist() + nuevas_columnas_reacciones)
df_columnas_reaccion['reactions'].fillna('', inplace=True)
df_columnas_reaccion = df_columnas_reaccion.reset_index(drop=True)



In [356]:
def merge_dict(lista):
    """Funci'on que une diccionarios en uno solo a travez de listas

    Args:
        lista (list): toma como argumdento una lista no vacia!!! que contenga varios diccionarios
    """
    new_dict = {'name':[], 'users':[]}

    for elemento in lista:
        new_dict['name'].append(elemento['name'])
        new_dict['users'].append(elemento['users'])

    return new_dict

In [ ]:
#
lista_dict = []

for i in df_columnas_reaccion.index:
    if df_columnas_reaccion['reactions'][i] != '':
        lista_dict.append(merge_dict(df_columnas_reaccion['reactions'][i]))
    else:
        lista_dict.append({'name': [], 'users': []})

df_reaction_provisoria = pd.DataFrame(lista_dict)

In [ ]:
#

simbolos_reaccion = ['white_check_mark', 'eyes', 'raised_hands']

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[0] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_finalizado'][i] = True

for i in df_reaction_provisoria.index:
    if simbolos_reaccion[1] in df_reaction_provisoria['name'][i]:
        df_columnas_reaccion['t_revisado'][i] = True

for i in df_reaction_provisoria.index:
    if any(simbolos_reaccion[2] in s for s in df_reaction_provisoria['name'][i]):
        df_columnas_reaccion['t_resuelto'][i] = True

df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']] = df_columnas_reaccion[['t_finalizado', 't_revisado', 't_resuelto']].fillna(value=False)

In [369]:
# Seleccionamos las columnas utiles del df y ademas se elimnaron las filas duplicadas y se ordenaron segun la fecha y hora de la creacion del thread. Antes de hacer este paso, debo sacar la info de la columna reactions 

df_clean = df_columnas_reaccion[['client_msg_id', 'text', 'user', 'ts', 'thread_ts', 'latest_reply', 't_finalizado', 't_revisado', 't_resuelto']].drop_duplicates().sort_values(['ts'])

df_clean = df_clean[df_clean['ts'] >= '1642171431.027300'].reset_index(drop=True)

In [371]:
# Columnas nuevas a agregar

nuevas_columnas = ['nuevo_req', 'ticket_ops', 'ticket_tech', 'urgente']

df_clean_new_columns = df_clean.reindex(columns=df_clean.columns.tolist() + nuevas_columnas)
df_clean_new_columns['latest_reply'].fillna('', inplace=True)

In [ ]:
# Se creo una lista con los simbolos asociado a las nuevas columnas.

simbolos = [':writing_hand:', ':dart:', ':robot_face:',':rotating_light:']

# Se itero el df para asociarle un valor de True or False a las respectivas columnas segun la simbologia del mensaje principal del thread.

for i in df_clean_new_columns.index:
    if df_clean_new_columns['latest_reply'][i] != '':
            for j in range(len(simbolos)):
                if simbolos[j] in df_clean_new_columns['text'][i]:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = True
                else:
                    df_clean_new_columns[df_clean_new_columns.columns[9 + j]][i] = False 

In [376]:
df_clean_new_columns.head(100)

,client_msg_id,text,user,ts,thread_ts,latest_reply,t_finalizado,t_revisado,t_resuelto,nuevo_req,ticket_ops,ticket_tech,urgente
0,85bc5361-aa7a-45c7-931c-b185d28298e2,*Resumen Simbología:*\n\n_*UPDATE:*_ \n _DEFIN...,U022N5F6ZGC,1642171431.027300,NaN,,False,False,False,NaN,NaN,NaN,NaN
1,89523989-32a4-4e4c-83ce-fed7c7c5f673,<@U02TWSY0V3M> tengo una duda con el mensaje d...,U02AJP0M7L1,1642173967.028200,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
2,f97780ee-fcc0-48e7-9648-ec84f6d86413,"Si, dejamos este espacio para que allí como in...",U02TWSY0V3M,1642174239.028400,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
3,3a1562c2-9dab-4cbd-b76b-a8b731c70c70,<@U02TWSY0V3M> ticket resuelto,U02AJP0M7L1,1642189849.029300,1642089212.013900,,True,False,True,NaN,NaN,NaN,NaN
4,a35833cc-50d2-4c28-ab9f-b5daad99a7c2,Gracias!,U02TWSY0V3M,1642192564.029700,1642089212.013900,,False,False,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,69004d36-d381-409d-9362-d2c77a3bd815,<@U02DBRSBBSM> ya se puede cerrar?,U02217J5895,1642687338.075600,1642082777.008400,,False,False,False,NaN,NaN,NaN,NaN
96,f918d022-96af-4570-b57f-9f6406b7f635,*Resuelto*,U022N5F6ZGC,1642687425.075900,1642428966.036600,,False,False,False,NaN,NaN,NaN,NaN
97,c683abef-458b-4d9a-9e01-7fc072929999,owner: <@U02KN0DBT9N> <@U02217J5895>\nJuan: Pl...,U02217J5895,1642687501.076100,1642483320.048400,,False,False,False,NaN,NaN,NaN,NaN
98,c76f4eb1-2488-4af1-8002-f0abefc83426,"Entiendo quedó lista <@U02G57HLWHK>, plis me c...",U02217J5895,1642687560.076300,1642603102.058500,,False,False,False,NaN,NaN,NaN,NaN
